In [ ]:
import requests
import os
import pandas as pd
import concurrent.futures
from bs4 import BeautifulSoup
import re

# Constants
BASE_URL = "https://www.sec.gov"
SEARCH_URL = "https://www.sec.gov/cgi-bin/browse-edgar"
HEADERS = {
    'Host': 'www.sec.gov',
    'Connection': 'close',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'X-Requested-With': 'XMLHttpRequest',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36',
}
OUTPUT_DIR = "Successful"
LOG_FILE = "edgar_download_log.txt"
EXCEL_FILE = "header_data.xlsx"

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)


def load_ciks(file_path):
    try:
        cik_data = pd.read_excel(file_path, sheet_name=None)
        ciks_df = cik_data[list(cik_data.keys())[0]]['cik']
        cleaned_ciks = ciks_df.apply(lambda x: x.replace('CIK', '').strip())
        return cleaned_ciks.tolist()
    except Exception as e:
        log_message(f"Error loading CIKs from file {file_path}: {e}")
        return []


def fetch_html(url):
    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        return response.text
    except requests.RequestException as e:
        log_message(f"Failed to fetch {url}: {e}")
        return None


def log_message(message):
    with open(LOG_FILE, 'a') as log_file:
        log_file.write(message + '\n')
    print(message)


def parse_filings(cik):
    url = f"{SEARCH_URL}?CIK={cik}&action=getcompany&type=S-1&S-1/A"
    html = fetch_html(url)
    if html:
        soup = BeautifulSoup(html, 'html.parser')
        filings = []
        table = soup.find('table', class_='tableFile2')
        if table:
            rows = table.find_all('tr')
            for row in rows[1:]:
                cells = row.find_all('td')
                if cells:
                    form_type = cells[0].text.strip()
                    if form_type in ["S-1", "S-1/A"]:
                        filing_href = BASE_URL + cells[1].a['href']
                        filing_date = cells[3].text.strip()
                        filings.append((filing_date, filing_href))
        return filings
    return []


def download_and_check_filing(filing_href, cik, header_data_list):
    doc_url = filing_href.replace("-index.htm", ".txt")
    response = requests.get(doc_url, headers=HEADERS)
    if response.status_code == 200:
        text_content = response.text
        header_data = extract_header_data(text_content)
        if header_data:
            header_data_list.append(header_data)
            file_name = f"{header_data['CIK']}_{header_data['FILED_AS_OF_DATE']}_{header_data['COMPANY_CONFORMED_NAME']}_{header_data['FORM_TYPE']}_{header_data['SEC_FILE_NUMBER']}_{header_data['FILM_NUMBER']}.txt"
            file_name = file_name.replace(" ", "_")
            save_path = os.path.join(OUTPUT_DIR, file_name)
            with open(save_path, 'wb') as file:
                file.write(response.content)
            log_message(f"Downloaded and saved as: {file_name}")
        else:
            log_message(f"Failed to extract header data from {doc_url}")
    else:
        log_message(f"Failed to download {doc_url}")


def extract_header_data(text_content):
    header_data = {}
    patterns = {
        'CIK': re.compile(r"CENTRAL INDEX KEY:\s+(\d+)"),
        'FILED_AS_OF_DATE': re.compile(r"FILED AS OF DATE:\s+(\d+)"),
        'COMPANY_CONFORMED_NAME': re.compile(r"COMPANY CONFORMED NAME:\s+(.*?)\n"),
        'FORM_TYPE': re.compile(r"FORM TYPE:\s+(.*?)\n"),
        'SEC_FILE_NUMBER': re.compile(r"SEC FILE NUMBER:\s+(\d+-\d+)"),
        'FILM_NUMBER': re.compile(r"FILM NUMBER:\s+(\d+)")
    }
    for key, pattern in patterns.items():
        match = pattern.search(text_content)
        if match:
            header_data[key] = match.group(1)
    return header_data if len(header_data) == 6 else None


def process_cik(cik, header_data_list):
    log_message(f"Processing CIK: {cik}")
    filings = parse_filings(cik)
    if filings:
        with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
            futures = [executor.submit(download_and_check_filing, filing[1], cik, header_data_list) for filing in filings]
            concurrent.futures.wait(futures)


def save_to_excel(header_data_list):
    df = pd.DataFrame(header_data_list)
    df.columns = ['cik', 'filed', 'spac', 'form', 'file number', 'film number']
    df.to_excel(EXCEL_FILE, index=False)
    log_message(f"Saved header data to {EXCEL_FILE}")


def main():
    file_path = 'path to your data file'
    ciks = load_ciks(file_path)
    if ciks:
        header_data_list = []
        with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
            futures = [executor.submit(process_cik, cik, header_data_list) for cik in ciks]
            concurrent.futures.wait(futures)
        save_to_excel(header_data_list)

if __name__ == "__main__":
    main()


Processing CIK: 0001326205
Processing CIK: 0001326190
Processing CIK: 0001723648
Processing CIK: 0000811222Processing CIK: 0001246713

Processing CIK: 0001298663
Processing CIK: 0001320716
Processing CIK: 0001331858
Processing CIK: 0001374346
Saved header data to header_data.xlsx
